<a href="https://colab.research.google.com/github/Smart-Sej/SimulatedAnnealing-GlobalPeakSearch/blob/main/Simulated_Annealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

# === CONFIG ===
N_POINTS = 100
X_RANGE = (-10, 10)
Y_RANGE = (-10, 10)
NOISE_STD = 0.3

def hidden_peak_function(x, y):
    z = (
        8 * np.exp(-((x - 2.73)**2 + (y + 4.58)**2) / 6) +
        10 * np.exp(-((x + 5.64)**2 + (y - 3.29)**2) / 8) +
        4 * np.sin(0.7 * x) * np.cos(0.5 * y)
    )
    return z

# === Data generation ===
def generate_training_data(n_points=N_POINTS):
    x = np.random.uniform(X_RANGE[0], X_RANGE[1], n_points)
    y = np.random.uniform(Y_RANGE[0], Y_RANGE[1], n_points)
    z = hidden_peak_function(x, y) + np.random.normal(0, NOISE_STD, n_points)
    df = pd.DataFrame({"x": x.round(2), "y": y.round(2), "z": z.round(2)})
    return df


In [ ]:
# submission.py
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import time
import random

class Model:
    def __init__(self):
        self.x = None
        self.y = None
        self.z = None
        self.last_paths = []  # Store paths explored during predict

    def fit(self, train_df: pd.DataFrame):
        self.x = train_df["x"].values
        self.y = train_df["y"].values
        self.z = train_df["z"].values
        self.xmin, self.xmax = self.x.min(), self.x.max()
        self.ymin, self.ymax = self.y.min(), self.y.max()

    def _f(self, xq, yq):
        val = griddata((self.x, self.y), self.z, (xq, yq), method="linear")
        if np.isnan(val):
            return -np.inf
        return val

    def _simulated_annealing(self,time_limit=115,step_size=0.5,cooling=0.995,record_path=False,start_point=None):
      """
      Find max height using Simulated Annealing within time constraints.
      Can optionally start at a given (x, y) instead of a random point.
      """
      start_time = time.time()

      # Initialize current point
      if start_point is not None:
          x_curr, y_curr = start_point
      else:
          x_curr = np.random.uniform(self.xmin, self.xmax)
          y_curr = np.random.uniform(self.ymin, self.ymax)

      z_curr = self._f(x_curr, y_curr)
      best_x, best_y, best_z = x_curr, y_curr, z_curr
      temp = 1.0

      # Initialize path recording if needed
      path = [(x_curr, y_curr, z_curr)] if record_path else None
      no_improve_steps=0

      while time.time() - start_time < time_limit:
          # Generate neighbor
          x_new = x_curr + np.random.uniform(-step_size, step_size)
          y_new = y_curr + np.random.uniform(-step_size, step_size)

          # Stay inside terrain bounds
          if not (self.xmin <= x_new <= self.xmax and self.ymin <= y_new <= self.ymax):
              continue

          z_new = self._f(x_new, y_new)
          if z_new == -np.inf:
              continue

          delta = z_new - z_curr

          # Accept new position probabilistically
          if delta > 0 or np.random.rand() < np.exp(delta / temp):
              x_curr, y_curr, z_curr = x_new, y_new, z_new
              # Check if new best
              if z_curr > best_z:
                  best_x, best_y, best_z = x_curr, y_curr, z_curr
                  no_improve_steps = 0  # reset counter
              else:
                  no_improve_steps += 1
          else:
              no_improve_steps += 1

          # Early stopping if no improvement
          if no_improve_steps >= 200:
              break
          temp *= cooling
          if temp < 1e-5:
              temp = 1.0  # occasional restart

          if record_path:
              path.append((x_curr, y_curr, z_curr))

      if record_path:
          return best_x, best_y, best_z, np.array(path)
      else:
          return best_x, best_y, best_z


    def predict(self) -> tuple[float, float, float]:
        xmid = (self.xmin + self.xmax)/2
        ymid = (self.ymin + self.ymax)/2

        starts = [
            (np.random.uniform(self.xmin, xmid), np.random.uniform(self.ymin, ymid)),
            (np.random.uniform(xmid, self.xmax), np.random.uniform(self.ymin, ymid)),
            (np.random.uniform(self.xmin, xmid), np.random.uniform(ymid, self.ymax)),
            (np.random.uniform(xmid, self.xmax), np.random.uniform(ymid, self.ymax))
        ]

        self.last_paths = []
        results = []
        for sp in starts:
            best_x, best_y, best_z, path = self._simulated_annealing(time_limit=20, record_path=True, start_point=sp)
            results.append((best_x, best_y, best_z))
            self.last_paths.append(path)

        best = max(results, key=lambda r: r[2])
        return float(best[0]), float(best[1]), float(best[2])



In [ ]:
# visualize_sa.py
import plotly.graph_objects as go
def actual_max(train_df):
    m=0
    for i in train_df['z'].values:
      if m<i:
        m=i

    return m

def visualize_sa_paths(model):
    """
    Visualize the SA paths stored in model.last_paths.
    """
    if not hasattr(model, "last_paths") or not model.last_paths:
        raise ValueError("No SA paths found. Call model.predict() first.")

    x, y, z = model.x, model.y, model.z

    fig = go.Figure(
        data=[
            go.Mesh3d(
                x=x, y=y, z=z,
                intensity=z,
                colorscale="Earth",
                opacity=0.8,
                name="Terrain",
            )
        ]
    )

    for i, path in enumerate(model.last_paths):
        fig.add_trace(go.Scatter3d(
            x=path[:, 0],
            y=path[:, 1],
            z=path[:, 2],
            mode="lines+markers",
            line=dict(color="red", width=4),
            marker=dict(size=3, color="red"),
            name=f"SA Path {i+1}"
        ))

    fig.update_layout(
        title="Simulated Annealing Search Paths",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        width=950,
        height=750
    )

    fig.show()


In [ ]:
if __name__ == "__main__":
    train_df = generate_training_data()
    train_df.to_csv("train.csv", index=False)
    print("Training dataset saved as train.csv")
    train_df = pd.read_csv('train.csv')
    model = Model()
    model.fit(train_df)

    best_x,best_y,best_z=model.predict()
    print("Best x,y,z coodrinates predicted: ",best_x,best_y,best_z)
    print("Efficiency: ",best_z/actual_max(train_df))
    visualize_sa_paths(model)


Training dataset saved as train.csv
Best x,y,z coodrinates predicted:  -4.987438673505245 3.209834718644712 9.731692404032467
Efficiency:  0.99302983714617
